In [ ]:
from shapely.geometry import Point, Polygon
import geopandas as gpd
import osmnx as ox
import matplotlib as mpl
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os.path import exists, join
from descartes import PolygonPatch
import folium
from tqdm import tqdm, tqdm_pandas
from geopy.distance import vincenty
from numpy import dot, arccos, sign, cross, degrees
from numpy.linalg import norm
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm, leave=True)

from pathlib import Path
from city_blocks import load_street_graph, remove_deadends, city_blocks, tmp
%matplotlib inline  


# To Do 
- calculate city block nodes using osmxn simplified graph, then get city block edges from non-simplified graph (original street graph). Using simplified graph will remedy the problem of recursion depth limit, as well as shortening the runtime. Getting the edges from the original graph will give city blocks with a smooth path. 
 - for each vertex in the simplest graph, find the nodes from the original street graph that are part of the vertex
 

## Settings

In [ ]:
### Settings for 
radius = 500
network_type = 'drive'
coords_nakano = (35.7059402,139.6664317)



### SPEED-UP SETTINGS
remove_road_curves = False
remove_dead_ends = True
cityblock_use_cached = False


### Load street graph
Use cached data if available, otherwise download first and then save to disk

In [ ]:
data_prefix = '{}-{}-{}'.format(coords_nakano, radius, network_type)

street_graph = load_street_graph(coords=coords_nakano, 
                                 radius=radius, 
                                 network_type=network_type,
                                 filename=data_prefix+'.graphml')
ox.plot_graph(street_graph);

## Simplify graph using `osmnx` simplification
Removes nodes to join edges that make up road curves. Note that doing this will affect the resulting city blocks. This is can be ammended by a different implementation. 

In [ ]:
if remove_road_curves:
    street_graph = ox.simplify_graph(street_graph)

In [ ]:
ox.plot_graph(street_graph);

## Simplify graph by removing dead-end roads
Removing the dead-end roads does not affect the shape of the city blocks. 

In [ ]:
if remove_dead_ends:
    street_graph = remove_deadends(street_graph)

We need to check that there's any street network left after applying our simplifications:

In [ ]:
assert len(street_graph) > 0

In [ ]:
ox.plot_graph(street_graph);

## Calculate city blocks

In [ ]:
areas = city_blocks(street_graph)

In [ ]:
areas.plot(figsize=(15,15));

In [ ]:
print('number of areas: {}'.format(len(areas)))

invalid = areas[areas.is_valid == False]
if not invalid.empty: 
    invalid.plot()

## Dealing with invalid polygons

When calculating city blocks for larger areas, it will often happen that some of the polygons representing the city blocks are invalid. This can be due to self-intersections. 

In [ ]:
print('number of areas after removing invalid polygons: {}'.format(len(areas)))

areas=areas[areas.geometry.is_valid]
areas.reset_index(inplace=True, drop=True)

Find MultiPolygons and extract simple polygons. Then create new dataframe with all polygons.

In [ ]:
multi_pol_idx = np.where(areas.geom_type == 'MultiPolygon')[0]
extra_pols = gpd.GeoDataFrame(
    {'geometry': [p for mp in areas.loc[multi_pol_idx].geometry for p in mp.geoms]})

# TO DO: add simple polygons created by splitting multipolygons to areas gdf

## Save areas as GeoJson

In [ ]:
geofile = Path(tmp, data_prefix).with_suffix('.geojson')

with geofile.open('w') as af: 
    af.write(areas.to_json())

Geojson file size in bytes:

In [ ]:
os.path.getsize(geofile.as_posix())